In [ ]:
import pypsa
import pandas as pd
import numpy as np

In [ ]:
solver = "cbc"

In [ ]:
# To calculate annualized capital expenditure
def calculate_annualised_capex(capex: float, interest: float, lifetime: int):
    crf = (
        interest * (1 + interest) ** lifetime / ((1 + interest) ** lifetime - 1)
    )  # Capital recovery factor
    return capex * crf

#### Initialize network

*** 
Check-point 1:

**Build a network in PyPSA with three nodes (`bus`) and following components to build a simple green hydrogen production system:**

1. An `Electricity` bus with electricity as `carrier`.
   1. A wind generator inside this bus.
   2. A solar generator inside this bus.
2. A `Hydrogen` bus with hydrogen as `carrier`.
   1. A hydrogen demand load inside this bus.
3. An electrolyser link to convert electricity into hydrogen.
4. A `Hydrogen_storage` bus with hydrogen as `carrier`.
   1. A hydrogen store inside this bus.
5. A bidirectional link to transfar the hydrogen between hydrogen and hydrogen storage buses.

**For simplicity, we assume the hydrogen demand profile to be flat for now. Afterwards, we want to supply electricy by attaching one renewable power plant implemented as (`generator`) (you have to call [`network.set_snapshots`](https://pypsa.readthedocs.io/en/latest/api/_source/pypsa.Network.set_snapshots.html) to select a year). As help you should have a look at the [PyPSA documentation](https://pypsa.readthedocs.io/en/latest/) and the [minimal lopf example](https://pypsa.readthedocs.io/en/latest/examples/lopf-with-heating.html), understand what the [components documentation](https://pypsa.readthedocs.io/en/latest/user-guide/components.html) of PyPSA gives you and that you can find the underlying objective function and constraints in the [LOPF documentation](https://pypsa.readthedocs.io/en/stable/user-guide/optimal-power-flow.html).**

> **Remarks:** For time reasons, you do not have to build the network from scratch. However, to get you acquainted with PyPSA we have omitted a few elements or some of the parameters of the network marked by three question marks `???`. Either, you have to add an element similar to the one in the box above or add a few parameters.

In [ ]:
# Create empty PyPSA network
network = pypsa.Network()

In [ ]:
# Set snapshots to the year 2023 and at hourly resolution
snapshots = pd.date_range("01-01-2023", "01-01-2024", freq="H", inclusive="left")
network.set_snapshots(snapshots)

In [ ]:
network.snapshots

In [ ]:
# Import an example of wind daily pattern
wind_pattern = pd.read_csv("../data/weather data/example_onshore_wind_daily_pattern.csv")["daily pattern"]
# annual time-series availability of onshore wind (just a simplified example)
wind_profile = pd.Series(list(wind_pattern)*365, index=network.snapshots)

In [ ]:
# Import an example of solar daily pattern
solar_pattern = pd.read_csv("../data/weather data/example_solar_daily_pattern.csv")["daily pattern"]
# annual time-series availability of solar (just a simplified example)
solar_profile = pd.Series(list(solar_pattern)*365, index=network.snapshots)

Add an `electricity` bus with electricity as `carrier`

In [ ]:
network.add(class_name="Bus", name="electricity", carrier="electricity")

Add a `hydrogen` bus with hydrogen as `carrier`

In [ ]:
network.add(class_name="Bus", name="hydrogen", carrier="hydrogen")

Add a `hydrogen_storage` bus with hydrogen as `carrier`

In [ ]:
network.add(class_name="Bus", name="hydrogen_storage", carrier="hydrogen")

In [ ]:
network.buses

Add a constant hourly hydrogen load of `100MW` at the hydrogen bus. The name of the load can be `hydrogen_load`

In [ ]:
network.add(class_name="Load", name="hydrogen_load", bus="hydrogen", p_set=100)

In [ ]:
network.loads

Add a store at the hydrogen_storage bus with a initial energy capacity of `0MWh`, and marginal cost of `0$/MWh` into the network? The name of the store can be `hydrogen_tank`. 

In [ ]:
network.add(
      class_name="Store",
      name="hydrogen_tank",
      bus="hydrogen_storage",
      carrier="hydrogen",
      e_nom_extendable=True,
      e_cyclic=True,
      marginal_cost=0, #$/MWh
      )

Add a bidirectional link, representing the hydrogen charging and discharging between hydrogen and hydrogen_storage with marginal cost of `0$/MWh` into the network? The name of the link can be `hydrogen_flow`. 

In [ ]:
network.add(
      class_name="Link",
      name="hydrogen_flow",
      bus0="hydrogen",
      bus1="hydrogen_storage",
      p_nom_extendable=True,
      marginal_cost=0, #$/MWh
      efficiency=1,
      p_min_pu=-1
      )

Add a wind generator at the electricity bus with a initial capacity of `100MW`, maximum capacity of `500MW`, based on provided CAPEX, FOM, VOM, efficiency, interest rate and lifetime? The name of the generator can be `onshore_wind`. 

> **Source:** all costs for the example are taken from [PyPSA technology database](https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2025.csv) and the assumptions in year 2023 in [EU map of hydrogen production costs](https://public.flourish.studio/visualisation/16659363/), with exchange rate of `1.1USD/EUR`

In [ ]:
# Onshore Wind's techno-economic parameters are given as:
eur_to_usd = 1.1
lifetime = 25
interest = 0.05

CAPEX = 1420*eur_to_usd # $/MW
FOM = 28*eur_to_usd  # $/MW fixed
VOM = 0 # $/MWh variable

efficiency = 1

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)

network.add(
    class_name="Generator", 
    name="onshore_wind",
    bus="electricity",
    carrier="electricity",
    p_nom_extendable=True,
    p_nom=100, # MW
    p_nom_max=500, # MW
    capital_cost=annualized_capex + FOM, #$/MW
    marginal_cost=VOM, #$/MWh
    efficiency=efficiency,
    lifetime=lifetime,
    p_max_pu=wind_profile
)

Add a solar generator at the electricity bus with a initial capacity of `10MW`, maximum capacity of `500MW`, based on provided CAPEX, FOM, VOM, efficiency, interest rate and lifetime? The name of the generator can be `solar`. 

> **Source:** all costs for the example are taken from [PyPSA technology database](https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2025.csv) and the assumptions in year 2023 in [EU map of hydrogen production costs](https://public.flourish.studio/visualisation/16659363/), with exchange rate of `1.1USD/EUR`

In [ ]:
# Solar's techno-economic parameters are given as:
eur_to_use = 1.1
lifetime = 20
interest = 0.05

CAPEX = 970*eur_to_use # $/MW
FOM = 16*eur_to_use  # $/MW fixed
VOM = 0 # $/MWh variable

efficiency = 1

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)

network.add(
    class_name="Generator", 
    name="solar",
    bus="electricity",
    carrier="electricity",
    p_nom_extendable=True,
    p_nom=50,
    p_nom_max=500, # MW
    capital_cost=annualized_capex + FOM, #$/MW
    marginal_cost=VOM, #$/MWh
    efficiency=efficiency,
    lifetime=lifetime,
    p_max_pu=solar_profile
)

In [ ]:
network.generators

Add an electrolyser link, representing the electrolysis conversion with initial capacity of `20MW`, based on provided CAPEX, FOM, VOM, efficiency, interest rate and lifetime.

> **Source:** all costs for the example are taken from [PyPSA technology database](https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2025.csv) and the assumptions in year 2023 in [EU map of hydrogen production costs](https://public.flourish.studio/visualisation/16659363/), with exchange rate of `1.1USD/EUR`

In [ ]:
# Electrolyser's techno-economic parameters are given as:
eur_to_usd = 1.1
lifetime = 10
interest = 0.05

CAPEX = 1590*eur_to_usd # $/MW
FOM = 32*eur_to_usd  # $/MW fixed
VOM = 0 # $/MWh variable
efficiency = 0.65

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)

network.add(
    class_name="Link",
    name="electrolyser",
    bus0="electricity",
    bus1="hydrogen",
    p_nom_extendable=True,
    p_nom=20,
    capital_cost=annualized_capex + FOM, #$/MW
    marginal_cost=VOM, #$/MWh
    efficiency=efficiency,
    lifetime=lifetime
)

Now try to solve your network

In [ ]:
# Solve network using cbc solver
network.optimize(pyomo=False, solver_name=solver)

Lets look at some results! Back to read the docs. How would you look at results?

In [ ]:
# Inspect the interaction of different power plants to supply loads
network.generators_t.p[:24*7].plot()

In [ ]:
network.links_t.p0.head()

In [ ]:
network.links_t.p1.head()

In [ ]:
# Exporting check-point 1 network
network.export_to_netcdf("../results/network_d2_e4-1.nc")

***
Check-point 2:

**Remove the electrolyser link from the previous exercise and replace it with a new electrolyser link including compressor with cost assumption and efficiency adjustment**

In [ ]:
# Import check-point 1 network
network = pypsa.Network("../results/network_d2_e4-1.nc")

In [ ]:
# remove hydrogen load from previous check-point
network.remove(class_name="Load", name="hydrogen_load")

In [ ]:
# remove electrolyser link from previous check-point
network.remove(class_name="Link", name="electrolyser")

Add a hourly hydrogen load at the hydrogen bus according to the daily pattern of industrial hydrogen demand (`100MW per hour` from 1am to 7pm, and with a shutodwn of 5 hours). The name of the load can be `hydroge_load`

**To set hourly hydrogen demand, you have to call [`network.set_snapshots`](https://pypsa.readthedocs.io/en/latest/api/_source/pypsa.Network.set_snapshots.html) to select a year.For help, you should have a look at the [PyPSA documentation](https://pypsa.readthedocs.io/en/latest/) and the [optimzation with Linopy](https://pypsa.readthedocs.io/en/latest/examples/optimization-with-linopy.html), understand what the [components documentation](https://pypsa.readthedocs.io/en/latest/user-guide/components.html) of PyPSA gives you and that you can find the underlying objective function and constraints in the [System Optimization documentation](https://pypsa.readthedocs.io/en/stable/user-guide/optimal-power-flow.html).**

In [ ]:
# Import an example of wind daily pattern
load_pattern = pd.read_csv("../data/weather data/example_industrial_h2_demand_daily_pattern.csv")["daily pattern"]
# annual time-series availability of onshore wind (just a simplified example)
load_profile = pd.Series(list(load_pattern)*365, index=network.snapshots)

In [ ]:
network.add(
    class_name="Load", 
    name="hydrogen_load", 
    bus="hydrogen", 
    p_set=load_profile
)

Add an electrolyser link including compressor, representing the electrolysis conversion with initial capacity of `20MW`, based on provided CAPEX, FOM, VOM, water_cost, efficiency, interest rate and lifetime.

> **Source:** all costs for the example are taken from [PyPSA technology database](https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2025.csv) and the assumptions in year 2023 in [EU map of hydrogen production costs](https://public.flourish.studio/visualisation/16659363/), with exchange rate of `1.1USD/EUR`

In [ ]:
# Electrolyser's techno-economic parameters are given as:
eur_to_usd = 1.1
lifetime = 10
interest = 0.05
water_cost = 1.8 # EUR/m3, seawater
water_conversion_cost= 21 # kgH2O/kgH2

CAPEX = 1590*eur_to_usd # $/MW
FOM = 32*eur_to_usd  # $/MW fixed
VOM = 0 # $/MWh variable

# calculate marginal cost for water consumption (EUR/MWh)
# =water_cost*water_converstion_cost/(33.33(kWh)/1000)
water_cost = ((water_cost/1000)*water_conversion_cost)/(33.33/1000)*eur_to_usd # 1kgH2 = 33.33kWh

efficiency = 0.65


# Adding compressor to compress hydrogen from electrolyser into hydrogen storage
# 1. Convert electricity consumption for compressor from kWh_el/kg_h2 into kWh_el/kWh_h2
# 2. the new value represents the share of electricity consumption, and the rest of share will be used for electrolyzer.
electricity_consumption = 0.6  # kWh/kgH2
elect_share = 1 - (electricity_consumption/33.33)
new_efficiency = efficiency*elect_share

# Compressor's techno-economic parameters are given as:
lifetime_com = 25
CAPEX_COM = 1720*eur_to_usd # $/MW
FOM_COM = 34*eur_to_usd  # $/MW fixed
VOM_COM = 0 # $/MWh variable

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)
annualized_capex_com = calculate_annualised_capex(CAPEX_COM, interest, lifetime_com)

network.add(
    class_name="Link",
    name="electrolyser",
    bus0="electricity",
    bus1="hydrogen",
    p_nom_extendable=True,
    p_nom=20,
    capital_cost=annualized_capex + FOM + annualized_capex_com + FOM_COM, #$/MW
    marginal_cost=VOM + water_cost + VOM_COM, #$/MWh
    efficiency=new_efficiency,
    lifetime=lifetime
)

Ending of check-point 2 - Solve network, analyse results and export network

In [ ]:
# Solve network using cbc solver
network.optimize(pyomo=False, solver_name=solver)

In [ ]:
# Inspect the interaction of different power plants to supply loads
network.generators_t.p[:24*7].plot()

In [ ]:
# Inspect the change of energy capacity in hydrogen_tank store to supply loads
network.stores_t.e[:24*7].plot()

In [ ]:
# Inspect the change of energy capacity in hydrogen_tank store to supply loads
network.stores_t.p[:24*7].plot()

In [ ]:
# Inspect the electrolysis conversion
network.links_t.p0["electrolyser"][:24*7].plot()

In [ ]:
# Inspect the charging flow
network.links_t.p0["hydrogen_flow"][:24*7].plot()

In [ ]:
# Exporting check-point 2 network
network.export_to_netcdf("../results/network_d2_e4-2.nc")